In [1]:
# IMPORT LIBRARIES

import numpy as np 
import sympy as sp
import matplotlib.pyplot as plt

In [2]:
# DATA 

T = 60 # Operating temperature [°C]
rho_H2O = 1 # Water density [g/cm3]
NAV = 6.022 * 10 ** 23 # Avogadro number [atoms/mol]
MM_H2O = 18.015 # Water molar mass [g/mol]
rho_235 = 19.1 # Uranium 235 density [g/cm3]
MM_235 = 235.044 # Uranium-235 molar mass [g/mol]
B_Core = 0.05624 # Core Buckling [1/cm]
nu = 2.44 # Neutrons per fission [-]
D_Core = 0.16 # Core diffusion coefficient [cm]
H = 50 # Slab width [cm]
q = 25 # Power density [W/cm3]
E_Fission = 211.5 * 10 ** 6  # Energy released by fission [eV]
conversion = 6.242 * 10 ** 18 #Conversion coefficient eV-J
E_Fission = E_Fission / conversion # Energy released by fission [J]

In [3]:
# FUNCTION AND SYMBOLIC VARIABLES DEFINITION

x = sp.symbols('x')
B = sp.symbols('B', nonnegative = True) 
L_c = sp.symbols('L_c', nonnegative = True)
L_r = sp.symbols('L_r', nonnegative = True)
D = sp.symbols('D', nonnegative = True) # D_R = d_C
H = sp.symbols('H', nonnegative = True) # Core width
F_c = sp.Function('phi_c', real = True)(x)
F_r = sp.Function('phi_r', real = True)(x)

In [4]:
# ODE + BC

ODE_c = sp.Eq(sp.diff(sp.diff(F_c, x)) + B** 2 * F_c, 0)
print('ODE FOR THE CORE:')
display(ODE_c)

ODE_r = sp.Eq(sp.diff(sp.diff(F_r, x)) - (1/L_r**2) * F_r, 0)
print('ODE FOR THE REFLECTOR:')
display(ODE_r)

flux_c = sp.dsolve(ODE_c)
sC1 = sp.symbols('C1')   
sC2 = sp.symbols('C2')
print('GENERAL SOLUTION FOR THE FLUX IN THE CORE:')
display(flux_c)

flux_r = sp.dsolve(ODE_r)
sC3 = sp.symbols('C3')
sC4 = sp.symbols('C4')
flux_r = flux_r.subs({sC1: sC3, sC2: sC4})
print('GENERAL SOLUTION FOR THE FLUX IN THE REFLECTOR:')
display(flux_r)

# Symmetry BC
symmetry = sp.Eq(
    sp.diff(flux_c.rhs, x).subs(x, 0), 
    0 
)
print('SYMMETRY CONDITION IN THE CORE:')
display(symmetry)

# Bounded BC (da migliorare, non so farla generica...)
flux_r = flux_r.subs(sC4, 0)
print('GENERAL SOLUTION FOR THE FLUX IN THE REFLECTOR AFTER BOUNDED BC:')
display(flux_r)

# Continuity BC
continuity_f = sp.Eq(
    flux_c.rhs.subs(x, H/2), 
    flux_r.rhs.subs(x, H/2)
)
print('CONTINUITY CONDITION FOR THE FLUX AT THE INTERFACE:')
display(continuity_f)
continuity_curr = sp.Eq(
    -D* sp.diff(flux_c.rhs, x).subs(x, H/2),
    -D * sp.diff(flux_r.rhs, x).subs(x, H/2)
    )
print('CONTINUITY CONDITION FOR THE CURRENT AT THE INTERFACE:')
display(continuity_curr)

matrix = sp.linear_eq_to_matrix([symmetry, continuity_f, continuity_curr, ], [sC1, sC2, sC3])[0]
det = sp.det(matrix)
print('DETERMINANT OF THE MATRIX OF COEFFICIENTS:') # Non so se è utile 
display(det)

ODE FOR THE CORE:


Eq(B**2*phi_c(x) + Derivative(phi_c(x), (x, 2)), 0)

ODE FOR THE REFLECTOR:


Eq(Derivative(phi_r(x), (x, 2)) - phi_r(x)/L_r**2, 0)

GENERAL SOLUTION FOR THE FLUX IN THE CORE:


...

Eq(C1*sin(B*H/2) + C2*cos(B*H/2), C3*exp(-H/(2*L_r)))

CONTINUITY CONDITION FOR THE CURRENT AT THE INTERFACE:


Eq(-D*(B*C1*cos(B*H/2) - B*C2*sin(B*H/2)), C3*D*exp(-H/(2*L_r))/L_r)

DETERMINANT OF THE MATRIX OF COEFFICIENTS:


(B**2*D*L_r*exp(-H/(2*L_r))*sin(B*H/2) - B*D*exp(-H/(2*L_r))*cos(B*H/2))/L_r